In [1]:
#baseLine
#标题无法分出四字词组
#标题的分词结果缺乏在正文中的合理位置
import pandas as pd
import re
import jieba.analyse

In [2]:
raw_data=pd.read_csv('data/all_docs.txt',sep='\001',header=None)
raw_cut=pd.read_csv('data/fenci_doc.txt',sep='\001',header=None)
raw_data.columns=['id','title','doc']
raw_data['article_cut']=raw_cut

train_data=pd.read_csv('data/train_docs_keywords.txt',sep='\t')
train_data.columns=['id','label']
train_id_list=list(train_data['id'].unique())
train_title_doc=raw_data[raw_data['id'].isin(train_id_list)]
train_title_doc=pd.merge(train_title_doc,train_data,on=['id'],how='inner')

test_title_doc=raw_data[~raw_data['id'].isin(train_id_list)]


In [21]:
train_title_doc['title_cut']=train_title_doc['title'].apply(lambda x:','.join(jieba.analyse.extract_tags(x,topK=4,allowPOS=('ns','nz','nr','vn','n'))))
train_title_doc['doc_cut']=train_title_doc['doc'].apply(lambda x:','.join(jieba.analyse.extract_tags(x,topK=4,allowPOS=('ns','nz','nr','vn','n'))))

In [12]:
test_title_doc['title']=test_title_doc['title'].astype(str)
test_title_doc['article_cut']=test_title_doc['article_cut'].astype(str)
test_title_doc['title_regex'] = test_title_doc['title'].apply(lambda x:','.join(re.findall(r"《(.+?)》",x)))
test_title_doc['title']=test_title_doc['title'].apply(lambda x:''.join(re.split(r"《.+?》",x)))
test_title_doc['title']=test_title_doc['title'].apply(lambda x:''.join(filter(lambda ch:ch not in '1234567890',x)))
test_title_doc['title_cut']=test_title_doc['title'].apply(lambda x:','.join(jieba.analyse.extract_tags(x,topK=4,allowPOS=('ns','nz','nr','vn','n'))))
test_title_doc['doc_cut']=test_title_doc['doc'].apply(lambda x:','.join(jieba.analyse.extract_tags(x,topK=4,allowPOS=('ns','nz','nr','vn','n'))))
test_title_doc['title_cut']=test_title_doc['title'].apply(lambda x:','.join(jieba.analyse.extract_tags(x,topK=4,allowPOS=('ns','nz','nr','vn','n'))))
test_title_doc['all_cut']=test_title_doc['doc_cut']+test_title_doc['title_cut']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [37]:
import jieba.posseg as pseg

str1='退票'
word0=pseg.cut(str1)
for word,flag in word0:
    print(word+':'+flag)

退票:v


In [14]:
import gensim
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
 
TaggededDocument = gensim.models.doc2vec.TaggedDocument
test_title_doc_result=test_title_doc[['id','title_regex','title_cut','article_cut']]
model_dm = Doc2Vec.load("data/model_dm_20")

def cal_dis(vec1,vec2):
    dist = numpy.sqrt(numpy.sum(numpy.square(vec1 - vec2)))
    return dist/numpy.sqrt(200)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [15]:
import jieba.posseg as pseg
import numpy
label1=[]
label2=[]
id_list=[]
num=0
for i in test_title_doc_result.values:
    num=num+1
    iD=i[0]
    id_list.append(iD)
    title_regex=i[1].split(',')
    title_cut=i[2].split(',')
    article_cut=i[3].split(',')
    article_vector= model_dm.infer_vector(article_cut)
    min_index=0
    min_distance=1000
    #title_cut=clean_words(title_cut,pos_list)
    j=0
    for word in title_cut:
        words_list=[]
        words_list.append(word)
        word_vector=model_dm.infer_vector(words_list)
        if cal_dis(word_vector,article_vector)<min_distance:
            min_distance=cal_dis(word_vector,article_vector)
            min_index=j
        j=j+1
    secmin_distance=1000
    secmin_index=0
    j=0
    for word in title_cut:
        
        words_list=[]
        words_list.append(word)
        word_vector=model_dm.infer_vector(words_list)
        if cal_dis(word_vector,article_vector)<secmin_distance and cal_dis(word_vector,article_vector)>min_distance:
            secmin_distance=cal_dis(word_vector,article_vector)
            secmin_index=j
        j=j+1

    new_title_cut=[]
    for word in title_cut:
        word0=pseg.cut(word)
        for w in word0:
            if w.flag=='nr':
                new_title_cut.append(word)
    k=0
    for word in title_cut:
        if k==min_index :
            if word not in new_title_cut:
                new_title_cut.append(word)
        k=k+1
    k=0
    for word in title_cut:
        if k==secmin_index :
            if word not in new_title_cut:
                new_title_cut.append(word)
        k=k+1
    for word in title_cut:
        if word not in new_title_cut:
            new_title_cut.append(word)
   
    if title_regex[0]=='':
        temp_result=new_title_cut
    else:
        temp_result=title_regex+new_title_cut
    if len(temp_result)>1:
        label1.append(temp_result[0])
        label2.append(temp_result[1])
    elif len(temp_result)==1:
        label1.append(temp_result[0])
        label2.append('')
    else:
        label1.append('')
        label2.append('')
    print('finishing %d articles'%num)
    
test_result=pd.DataFrame();
test_result['id']=id_list
test_result['label1']=label1
test_result['label1'] = test_result['label1'].replace('','nan')
test_result['label2']=label2
test_result['label2'] = test_result['label2'].replace('','nan')


finishing 1 articles
finishing 2 articles
finishing 3 articles
finishing 4 articles
finishing 5 articles
finishing 6 articles
finishing 7 articles
finishing 8 articles
finishing 9 articles
finishing 10 articles
finishing 11 articles
finishing 12 articles
finishing 13 articles
finishing 14 articles
finishing 15 articles
finishing 16 articles
finishing 17 articles
finishing 18 articles
finishing 19 articles
finishing 20 articles
finishing 21 articles
finishing 22 articles
finishing 23 articles
finishing 24 articles
finishing 25 articles
finishing 26 articles
finishing 27 articles
finishing 28 articles
finishing 29 articles
finishing 30 articles
finishing 31 articles
finishing 32 articles
finishing 33 articles
finishing 34 articles
finishing 35 articles
finishing 36 articles
finishing 37 articles
finishing 38 articles
finishing 39 articles
finishing 40 articles
finishing 41 articles
finishing 42 articles
finishing 43 articles
finishing 44 articles
finishing 45 articles
finishing 46 articl

finishing 374 articles
finishing 375 articles
finishing 376 articles
finishing 377 articles
finishing 378 articles
finishing 379 articles
finishing 380 articles
finishing 381 articles
finishing 382 articles
finishing 383 articles
finishing 384 articles
finishing 385 articles
finishing 386 articles
finishing 387 articles
finishing 388 articles
finishing 389 articles
finishing 390 articles
finishing 391 articles
finishing 392 articles
finishing 393 articles
finishing 394 articles
finishing 395 articles
finishing 396 articles
finishing 397 articles
finishing 398 articles
finishing 399 articles
finishing 400 articles
finishing 401 articles
finishing 402 articles
finishing 403 articles
finishing 404 articles
finishing 405 articles
finishing 406 articles
finishing 407 articles
finishing 408 articles
finishing 409 articles
finishing 410 articles
finishing 411 articles
finishing 412 articles
finishing 413 articles
finishing 414 articles
finishing 415 articles
finishing 416 articles
finishing 4

finishing 743 articles
finishing 744 articles
finishing 745 articles
finishing 746 articles
finishing 747 articles
finishing 748 articles
finishing 749 articles
finishing 750 articles
finishing 751 articles
finishing 752 articles
finishing 753 articles
finishing 754 articles
finishing 755 articles
finishing 756 articles
finishing 757 articles
finishing 758 articles
finishing 759 articles
finishing 760 articles
finishing 761 articles
finishing 762 articles
finishing 763 articles
finishing 764 articles
finishing 765 articles
finishing 766 articles
finishing 767 articles
finishing 768 articles
finishing 769 articles
finishing 770 articles
finishing 771 articles
finishing 772 articles
finishing 773 articles
finishing 774 articles
finishing 775 articles
finishing 776 articles
finishing 777 articles
finishing 778 articles
finishing 779 articles
finishing 780 articles
finishing 781 articles
finishing 782 articles
finishing 783 articles
finishing 784 articles
finishing 785 articles
finishing 7

finishing 1101 articles
finishing 1102 articles
finishing 1103 articles
finishing 1104 articles
finishing 1105 articles
finishing 1106 articles
finishing 1107 articles
finishing 1108 articles
finishing 1109 articles
finishing 1110 articles
finishing 1111 articles
finishing 1112 articles
finishing 1113 articles
finishing 1114 articles
finishing 1115 articles
finishing 1116 articles
finishing 1117 articles
finishing 1118 articles
finishing 1119 articles
finishing 1120 articles
finishing 1121 articles
finishing 1122 articles
finishing 1123 articles
finishing 1124 articles
finishing 1125 articles
finishing 1126 articles
finishing 1127 articles
finishing 1128 articles
finishing 1129 articles
finishing 1130 articles
finishing 1131 articles
finishing 1132 articles
finishing 1133 articles
finishing 1134 articles
finishing 1135 articles
finishing 1136 articles
finishing 1137 articles
finishing 1138 articles
finishing 1139 articles
finishing 1140 articles
finishing 1141 articles
finishing 1142 a

finishing 1445 articles
finishing 1446 articles
finishing 1447 articles
finishing 1448 articles
finishing 1449 articles
finishing 1450 articles
finishing 1451 articles
finishing 1452 articles
finishing 1453 articles
finishing 1454 articles
finishing 1455 articles
finishing 1456 articles
finishing 1457 articles
finishing 1458 articles
finishing 1459 articles
finishing 1460 articles
finishing 1461 articles
finishing 1462 articles
finishing 1463 articles
finishing 1464 articles
finishing 1465 articles
finishing 1466 articles
finishing 1467 articles
finishing 1468 articles
finishing 1469 articles
finishing 1470 articles
finishing 1471 articles
finishing 1472 articles
finishing 1473 articles
finishing 1474 articles
finishing 1475 articles
finishing 1476 articles
finishing 1477 articles
finishing 1478 articles
finishing 1479 articles
finishing 1480 articles
finishing 1481 articles
finishing 1482 articles
finishing 1483 articles
finishing 1484 articles
finishing 1485 articles
finishing 1486 a

...

finishing 105926 articles
finishing 105927 articles
finishing 105928 articles
finishing 105929 articles
finishing 105930 articles
finishing 105931 articles
finishing 105932 articles
finishing 105933 articles
finishing 105934 articles
finishing 105935 articles
finishing 105936 articles
finishing 105937 articles
finishing 105938 articles
finishing 105939 articles
finishing 105940 articles
finishing 105941 articles
finishing 105942 articles
finishing 105943 articles
finishing 105944 articles
finishing 105945 articles
finishing 105946 articles
finishing 105947 articles
finishing 105948 articles
finishing 105949 articles
finishing 105950 articles
finishing 105951 articles
finishing 105952 articles
finishing 105953 articles
finishing 105954 articles
finishing 105955 articles
finishing 105956 articles
finishing 105957 articles
finishing 105958 articles
finishing 105959 articles
finishing 105960 articles
finishing 105961 articles
finishing 105962 articles
finishing 105963 articles
finishing 10

finishing 106243 articles
finishing 106244 articles
finishing 106245 articles
finishing 106246 articles
finishing 106247 articles
finishing 106248 articles
finishing 106249 articles
finishing 106250 articles
finishing 106251 articles
finishing 106252 articles
finishing 106253 articles
finishing 106254 articles
finishing 106255 articles
finishing 106256 articles
finishing 106257 articles
finishing 106258 articles
finishing 106259 articles
finishing 106260 articles
finishing 106261 articles
finishing 106262 articles
finishing 106263 articles
finishing 106264 articles
finishing 106265 articles
finishing 106266 articles
finishing 106267 articles
finishing 106268 articles
finishing 106269 articles
finishing 106270 articles
finishing 106271 articles
finishing 106272 articles
finishing 106273 articles
finishing 106274 articles
finishing 106275 articles
finishing 106276 articles
finishing 106277 articles
finishing 106278 articles
finishing 106279 articles
finishing 106280 articles
finishing 10

finishing 106577 articles
finishing 106578 articles
finishing 106579 articles
finishing 106580 articles
finishing 106581 articles
finishing 106582 articles
finishing 106583 articles
finishing 106584 articles
finishing 106585 articles
finishing 106586 articles
finishing 106587 articles
finishing 106588 articles
finishing 106589 articles
finishing 106590 articles
finishing 106591 articles
finishing 106592 articles
finishing 106593 articles
finishing 106594 articles
finishing 106595 articles
finishing 106596 articles
finishing 106597 articles
finishing 106598 articles
finishing 106599 articles
finishing 106600 articles
finishing 106601 articles
finishing 106602 articles
finishing 106603 articles
finishing 106604 articles
finishing 106605 articles
finishing 106606 articles
finishing 106607 articles
finishing 106608 articles
finishing 106609 articles
finishing 106610 articles
finishing 106611 articles
finishing 106612 articles
finishing 106613 articles
finishing 106614 articles
finishing 10

finishing 106925 articles
finishing 106926 articles
finishing 106927 articles
finishing 106928 articles
finishing 106929 articles
finishing 106930 articles
finishing 106931 articles
finishing 106932 articles
finishing 106933 articles
finishing 106934 articles
finishing 106935 articles
finishing 106936 articles
finishing 106937 articles
finishing 106938 articles
finishing 106939 articles
finishing 106940 articles
finishing 106941 articles
finishing 106942 articles
finishing 106943 articles
finishing 106944 articles
finishing 106945 articles
finishing 106946 articles
finishing 106947 articles
finishing 106948 articles
finishing 106949 articles
finishing 106950 articles
finishing 106951 articles
finishing 106952 articles
finishing 106953 articles
finishing 106954 articles
finishing 106955 articles
finishing 106956 articles
finishing 106957 articles
finishing 106958 articles
finishing 106959 articles
finishing 106960 articles
finishing 106961 articles
finishing 106962 articles
finishing 10

finishing 107269 articles
finishing 107270 articles
finishing 107271 articles
finishing 107272 articles
finishing 107273 articles
finishing 107274 articles
finishing 107275 articles
finishing 107276 articles
finishing 107277 articles
finishing 107278 articles
finishing 107279 articles
finishing 107280 articles
finishing 107281 articles
finishing 107282 articles
finishing 107283 articles
finishing 107284 articles
finishing 107285 articles
finishing 107286 articles
finishing 107287 articles
finishing 107288 articles
finishing 107289 articles
finishing 107290 articles
finishing 107291 articles
finishing 107292 articles
finishing 107293 articles
finishing 107294 articles
finishing 107295 articles
finishing 107296 articles


In [17]:
test_result.to_csv('data/jieba_doc2vec_result_0.csv',index=None)

In [18]:
result=pd.read_csv('data/jieba_doc2vec_result_0.csv')

In [19]:
result.head(50)

,id,label1,label2
0,D000001,林志颖,面膜
1,D000002,杨幂,太精彩
2,D000003,送祝福,无人
3,D000004,林心如,老公
4,D000005,颜值,NaN
5,D000006,范冰冰,公主
6,D000007,周迅,北京
7,D000008,张杰,废物
8,D000009,杨幂,杂志
9,D000010,杨幂,节目
